In [1]:
import cv2 as cv
import numpy as np

"""
定义裁减函数，四个参数分别是：
左上角横坐标x0
左上角纵坐标y0
裁剪宽度w
裁剪高度h

"""
crop_image = lambda img,x0,y0,w,h:img[y0:y0 + h, x0:x0 + w]


In [2]:
'''
随机裁剪
area_ratio 为裁剪画面占原画的比例
hw_vari 是扰动占原高宽比的比例范围

'''
def random_crop(img, area_ratio, hw_vari):
    h,w = img.shape[:2]
    hw_delta = np.random.uniform(-hw_vari,hw_vari)
    hw_mult = 1 + hw_delta
    
#     下标进行裁剪，宽高必须是整数
    w_crop = int(round(w * np.sqrt(area_ratio * hw_mult)))
#     裁剪宽度不能超过原有宽度
    if w_crop > w:
        w_crop = w
    
    h_crop = int(round(h * np.sqrt(area_ratio / hw_mult)))
    if h_crop > h:
        h_crop = h
#     随机生成左上角的位置 ++++++++
    x0 = np.random.random() * (w - w_crop)
    y0 = np.random.random() * (h - h_crop)
    
    return crop_image(img,x0,y0,w_crop,h_crop)

In [2]:
"""
定义旋转函数
angle 是逆时针旋转的角度
crop是个布尔值，表明是否要裁剪去除黑边

"""
def rotate_image(img,angle,crop):
    h,w = img.shape[:2]
    angle %= 360 
#     得到仿射矩阵
    affinemat = cv.getRotateMatrix2D((h/2, w/2), angle, 1)
    
    img_rotated = cv.warpAffine(img, affinemat, (w,h))
    
#     需要去除黑边
    if crop:
#     周期是180，且关于90对称
        angle_crop %= 180
        if angle_crop > 90:
            angle_crop = 180 - angle_crop
        theta = angle_crop / 180 * np.pi
#         计算裁剪系数
        hw_ratio = float(h) / float(w)
        
        numerator = np.cos(theta) + np.sin(theta) * np.tan(theta)
        r = hw_ratio if h > w else 1 / hw_ratio
        denominator = r * np.tan(theta) + 1
        crop_mult = numerator / denominator
        
#         得到裁剪区域
        w_crop = int(round(crop_mult * w))
        h_crop = int(round(crop_mult * h))
        x0 = int((w - w_crop) / 2)
        y0 = int((h - h_crop) / 2)
        img_rotated = crop_image(img_rotated , x0, y0, w_crop, h_crop)
    return img_rotated


In [3]:
"""
随机旋转
angle_vari 是旋转角度的范围[-angle_vari,angle_vari]
p_crop 是要进行曲黑边裁剪的比例

"""
def random_rotate(img , angle_vari, p_crop):
    angle = np.random.uniform(-angle_vari, angle_vari)
    crop = False if np.random.random() > p_crop else True
    return rotate_image(img, angle, crop)


In [5]:
"""
定义hsv 变换函数
hue_delta 是色调变化比例
sat_delta 是饱和度变化比例
val_delta 是明度变化比例

"""
def hsv_transform(img, hue_delta, sat_mult, val_mult):
#     ++++++
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    img_hsv[:,:,0] = (img_hsv[:,:,0] + hue_delta) % 180
    img_hsv[:,:,1] = img_hsv[:,:,1] * (1 + sat_mult)
    img_hsv[:,:,2] = img_hsv[:,:,2] * (1 + val_mult)
    
    img_hsv[img_hsv > 255] = 255
#   +++++++++
    return cv.cvtColor(img_hsv,cv.HSV2BGR)

In [7]:
"""
随机hsv 变换
hue_vari 
sat_vari
val_vari

"""
def random_hsv(img, hue_vari, sat_vari, val_vari):
    hue_delta = np.random.randint(-hue_vari, hue_vari)
    sat_mult = 1 + np.random.uniform(-sat.vari, sat.vari)
    val_mult = 1 + np.random.uniform(-val_vari, val.vari)
    
    return hsv_tranform(hue_delta, sat_mult, val_mult)

In [8]:
"""
定义gamma 函数
gamma 就是Gamma

"""
def gamma_transform(img, gamma):
    gamma_table = [np.power(x / 255, gamma)* gamma for x in range(256)]
#   ++++++++++++++=
    gamma_table = int(round(np.array(gamma_table)))
    
    return cv.LUT(img, gamma_table)


In [10]:
"""
随机gamma 变换
gamma_vari是gamma的变换范围[1/gamma_vari, gamma_vari]

"""
def random_gamma_transform(img, gamma_vari):
    log_gamma_vari = np.log(gamma_vari)
    alpha = np.random.uniform(-log_gamma_vari, log_gamma_vari)
    gamma = np.exp(alpha)
    return gamma_transform(img, gamma)